
![desc](Screenshot%20from%202021-05-19%2012-05-09.png)


![desc2](Screenshot%20from%202021-05-19%2012-05-28.png)

In [1]:
# installing required packages
import sys
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install glob2
!{sys.executable} -m pip install opencv-python

You should consider upgrading via the 'c:\users\david\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\david\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\david\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [1]:
# IMPORTS
import pandas as pd
import numpy as np

import os

import tensorflow
import glob
import cv2


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, InputLayer, Input, Dropout

from tensorflow.keras.applications import vgg16 
from tensorflow.keras.applications import vgg19 
from tensorflow.keras.applications import resnet50
from tensorflow.keras.applications import inception_v3

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import SGD, RMSprop, Adam

from tensorflow.keras.optimizers.schedules import InverseTimeDecay

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator




In [3]:
# enable NVIDIA CUDA/GPU usage
# NVIDIA CUDA must be installed beforehand
# (this is not a necessary step)

gpu = tensorflow.config.list_physical_devices('GPU')[0]
gpu

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')

In [ ]:
# DATA PREPARATION
"""
Here, we have two options. The first one is to go through all triplets, combine them into a single image, and then add this new image to a numpy array. 
In the end you return a complete array containing all new images (three in one combined), and use this array as your X.
The problem with this is that it is very memory intensive, since all images are in memory at once as your feature array.
Also it is less flexible. You specify the desired image size in the beginning, and then just have them like this.

Second option:
as before, go through all triplets and combine them into a single image. but instead returning them as numpy arrays, store these new images as actual image files, i.e. jpg, with a complete directory
structure etc.
Now we can use these new images very conveniently with keras' built in functions to load datasets from images/directories, we can also further resize them etc.
Upside: flexible and practical
Downside: you need a lot of disk space. for image size 300x400 (approx. the original image size), you would require ~22 GB of space.
"""

In [ ]:
# --------------------------------------------------------------------------
# OLD! FIRST OPTION (worse option). go to next cell for better way to create image features
# --------------------------------------------------------------------------
# DATA PREPARATION

# this function loads the three images and combines them into one for all triplet, and then returns them as a numpy array
# you can store the created numpy arrays with numpy functions directly to a file for later use, so only execute this function once and then just load from the file


# upside: you dont have to store images on disk
# downside: less flexible, you have the images as they are. Also, very memory intensive!


"""
    triplets_df - dataframe of triplets
    imgset - "train" or "test"
    height, width - what size to resize the images to (for a single image). the final size will be (height, width*3)
"""
def create_image_data_old(triplets_df,imgset="train", height=64,width=64):
   
    images = []
    labels = []
    
    for i in range(len(triplets_df.index.values)):
        
        # collect paths for the 3 images
        img_paths = []
        for j in range(3):
            img_path = str(triplets_df.iloc[i,j])
            for k in range(5-len(img_path)):
                img_path = "0" + img_path  # pad img_path with leading zeros
            img_paths.append("food/" + imgset + "/" + img_path + ".jpg")
       
        # combine the 3 images into 1 single image
        inputImages=[]
        outputImage = np.zeros((height,width*3,3), dtype="uint8")
        
        for imgpath in img_paths:
            image = cv2.imread(imgpath)
            image = cv2.resize(image, (width,height))
            inputImages.append(image)
            
        # problem: labels
        # in all our data, the first image is more similar to the second.
        # to get evenly distributed labels, we will just randomly swap the second and third picture (and therefore the label)
        
        label = np.random.randint(0,2)
        # label == 1 means first image is similar to second one, 0 means first similar to third one
        
        if imgset == "test":
            label = 1
            
            
        # depending on label, swap second and third image    
        outputImage[0:height, 0:width] = inputImages[0]
        outputImage[0:height, width:2*width] = inputImages[2-label]
        outputImage[0:height, 2*width:3*width] = inputImages[1+label]
        
        images.append(outputImage)
        labels.append(label)

        
    return np.array(images), np.array(labels)
    


    
    

In [4]:
# DATA PREPARATION & ORGANIZATION
# this function is only executed ONCE to create the img data directory structure, and to create and save the images that will be used as features
# it goes through all the triplets, combines the 3 images into one and saves them as a jpg.
# all necessary folders, also "target_directory", are created automatically
# train images are directly split into a train and a valid set, according to the fraction "validation_size"
# parameters "width" and "height" are used to resize the images. they correspond to the dimensions of a single image, so the final image will have size (height, width*3)! 
# The original images are around 300 x 400 in size.

# this requires quite a lot of disk space (22 GB with width 400 and height 300)!

# the original single images are assumed to be stored in subfolders food/train/ and food/test/

"""
    train_data - dataframe of train triplets
    test_data - dataframe of test triplets
    target_directory - directory name, will be created and the new images will be stored inside
    width - desired width of a single image
    height - desired height of a single image
    validation_size - fraction of the train images that will be put aside to use as a validation set
"""
def create_image_data(train_data, test_data,target_directory='img',width=64,height=64,validation_size=0.2):

    
    if os.path.isdir(target_directory):
        print("target directory already exists.")
        return
    
    #create directories, for every set split the images into class-subfolders (0 and 1)
    os.makedirs(target_directory + '/train/0')
    os.makedirs(target_directory + '/train/1')
    os.makedirs(target_directory + '/valid/0')
    os.makedirs(target_directory + '/valid/1')
    os.makedirs(target_directory + '/test/unknown')

    # split train images into train and valid set
    train, valid = train_test_split(train_data,test_size=validation_size)
    data = {
        'train': train,
        'valid': valid,
        'test': test_data
    }
    for dataset in data:
        written = 0
        labels = []
        for i in range(len(data[dataset].index.values)):
            imgs = []
            # load the three images and resize
            for j in range(3):
                img_path = str(data[dataset].iloc[i,j])
                for k in range(5-len(img_path)):
                    img_path = "0" + img_path  # pad img_path with leading zeros
                imgs.append(cv2.imread("food/" + ('test' if dataset == 'test' else 'train') + "/" + img_path + ".jpg"))
                imgs[j] = cv2.resize(imgs[j], (width,height))

            # problem: labels
            # in all our data, the first image is more similar to the second.
            # to get evenly distributed labels, we will just randomly swap the second and third picture (and therefore the label)

            label = np.random.randint(0,2)
            
            # label == 1 means first image is similar to second one, 0 means first similar to third one
            
            # this label swapping is only required for train and valid set
            if dataset == "test":
                label = 1


            # swap
            if label == 0:
                temp = imgs[1]
                imgs[1] = imgs[2]
                imgs[2] = temp



            labels.append(label)

            # concatenate the 3 images horizontally
            out = cv2.hconcat(imgs)
            
            path = target_directory + '/' + dataset + '/' + ('unknown' if dataset == 'test' else str(label)) + '/' + str(i) + '.jpg'

            # save image
            if cv2.imwrite(path, out):
                written += 1
            else:
                print('error when writing image')

                

        if dataset != 'test':
            np.save(target_directory+'/' + dataset + '_labels', np.array(labels))
        print("--------------------------")
        print(str(written) + " images written to " + dataset)
    
            

In [5]:
# EXECUTE THIS FUNCTION ONLY ONCE TO CREATE THE NEW IMAGE FEATURES (Create and save images)
# (second option, creating and storing new image files)
width = 400
height = 300


train_triplets = pd.read_csv('train_triplets.csv',sep=' ',header=None)
test_triplets = pd.read_csv('test_triplets.csv',sep=' ',header=None)


create_image_data(train_triplets, test_triplets,'img_400x300',width=width,height=height,validation_size=0.2)

--------------------------
47612 images written to train
--------------------------
11903 images written to valid
--------------------------
59544 images written to test


In [15]:
# Model Hyperparameters

# stored images will be resized to this width/height (per single image, so full image will have size (height, width*3))
width = 200
height = 150

# batch size
bs = 10
# epochs
ep = 50

# increase this if you want to remove more layers of the pretrained vision model
remove_layers = 1 # default = 1

# increase this if you want to set layers of the pretrained vision model as "trainable"
layers_trainable = 1 # default = 1

# activation function
dense_layers_activation = 'relu' # default = 'relu'


# pretrained model: VGG16, VGG19, ResNet50, InceptionV3, ...
pretrained = vgg16.VGG16(weights='imagenet',include_top=False, input_shape=(height,3*width, 3))
preprocess = vgg16.preprocess_input
decode = vgg16.decode_predictions


# optimizer
opt = 'adam' # adam, SGD, RMSprop...


In [16]:
# TRANSFER LEARNING

# use a pretrained vision model to extract useful features from the images, then use these features for binary classification with a new model/new layers

# question: should add new convolutional layers, or only Dense (fully connected) layers?
# i.e. should we try to detect new image features, or just use the ones we have and directly go to binary classification on these features?






model = pretrained

# add new classifier layers
flat1 = Flatten()(model.layers[-remove_layers].output)
class1 = Dense(1024, activation=dense_layers_activation)(flat1)
#class2 = Dense(128, activation=dense_layers_activation)(class1)
#dropout = Dropout(0.2)(class1) # dropout helps with overfitting (but we dont have overfitting yet :( )


output = Dense(1, activation='sigmoid')(class1) # equivalent or better than using 2 output neurons with softmax activation

# set layers as trainable
for layer in model.layers[:-layers_trainable]:
    layer.trainable = False


# create new model
model = Model(inputs=model.inputs, outputs=output)

    
# summarize
model.summary()



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 200, 900, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 900, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 900, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 450, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 450, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 450, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 50, 225, 128)      0   

In [17]:
# compile model
model.compile(loss='binary_crossentropy',optimizer=opt,metrics='accuracy')


In [18]:
# LOAD DATA
# assuming new image files have been created

# directory where the images have been stored
source_dir = 'img_400x300'


#y_train = np.load(source_dir + '/train_labels.npy')
#y_valid = np.load(source_dir + '/valid_labels.npy')

# classes are inferred from the folder structure

train_path = source_dir + '/train'
valid_path = source_dir + '/valid'
test_path = source_dir + '/test'

# create data generators
train_batches = ImageDataGenerator(preprocessing_function=preprocess).flow_from_directory(directory=train_path, target_size=(height,3*width), classes=None,class_mode='binary',batch_size=bs)
valid_batches = ImageDataGenerator(preprocessing_function=preprocess).flow_from_directory(directory=valid_path, target_size=(height,3*width), classes=None,class_mode='binary',batch_size=bs)
test_batches = ImageDataGenerator(preprocessing_function=preprocess).flow_from_directory(directory=test_path, target_size=(height,3*width), classes=None,class_mode=None, batch_size=bs, shuffle=False)


Found 47612 images belonging to 2 classes.
Found 11903 images belonging to 2 classes.
Found 59544 images belonging to 1 classes.


In [ ]:
# plot a batch of images. keep in mind that they are already preprocessed, so the colors look strange
import matplotlib.pyplot as plt


def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()
    
imgs, labels = next(train_batches)
plotImages(imgs)
print(labels)

In [19]:
# train the model
history = model.fit(x=train_batches,verbose=1,epochs=ep,steps_per_epoch=len(train_batches),validation_data=valid_batches,validation_steps=len(valid_batches))

Epoch 1/50
4762/4762 [==============================] - 797s 167ms/step - loss: 2.0839 - accuracy: 0.6038 - val_loss: 0.7582 - val_accuracy: 0.5747
Epoch 2/50
4762/4762 [==============================] - 783s 164ms/step - loss: 0.6327 - accuracy: 0.6591 - val_loss: 0.6903 - val_accuracy: 0.6580
Epoch 3/50
4762/4762 [==============================] - 774s 162ms/step - loss: 0.5761 - accuracy: 0.7007 - val_loss: 0.6188 - val_accuracy: 0.6690
Epoch 4/50
4762/4762 [==============================] - 774s 162ms/step - loss: 0.5108 - accuracy: 0.7287 - val_loss: 0.6823 - val_accuracy: 0.6668
Epoch 5/50
4762/4762 [==============================] - 774s 162ms/step - loss: 0.4793 - accuracy: 0.7459 - val_loss: 0.6644 - val_accuracy: 0.6703
Epoch 6/50
4762/4762 [==============================] - 774s 162ms/step - loss: 0.4578 - accuracy: 0.7612 - val_loss: 0.7906 - val_accuracy: 0.6879
Epoch 7/50
4762/4762 [==============================] - 774s 162ms/step - loss: 0.4457 - accuracy: 0.7725 - val_

NameError: name 'pretty_good' is not defined

In [10]:
# we're not here yet :(
y_pred = model.predict(test_batches)

In [13]:
y_pred = np.round(y_pred)

In [15]:
# finalize submission

f = open('submission.txt','w+')
for y in y_pred:
    f.write(str(int(y[0])) + '\n')
f.close()